<a href="https://colab.research.google.com/github/kunalkukreja21/jigsaw-rate-severity-of-toxic-comments/blob/main/jigsaw_rate_severity_of_toxic_comments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [ ]:
!pip install transformers sentencepiece datasets

     |████████████████████████████████| 3.4 MB 4.2 MB/s 
     |████████████████████████████████| 1.2 MB 44.2 MB/s 
     |████████████████████████████████| 306 kB 48.1 MB/s 
     |████████████████████████████████| 596 kB 50.8 MB/s 
     |████████████████████████████████| 895 kB 47.4 MB/s 
     |████████████████████████████████| 3.3 MB 28.6 MB/s 
     |████████████████████████████████| 61 kB 484 kB/s 
     |████████████████████████████████| 1.1 MB 49.5 MB/s 
     |████████████████████████████████| 132 kB 51.7 MB/s 
     |████████████████████████████████| 243 kB 50.6 MB/s 
     |████████████████████████████████| 160 kB 50.2 MB/s 
     |████████████████████████████████| 271 kB 55.4 MB/s 
     |████████████████████████████████| 192 kB 52.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import pandas as pd
import numpy as np

from transformers import AutoTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
import torch

import os

from sklearn.model_selection import train_test_split
from datasets import load_dataset
from datasets import Dataset

In [ ]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=1)
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [ ]:
# Split ruddit into train and test

def tokenize_function(examples):
    return tokenizer(examples["txt"], padding="max_length", truncation=True)

def prepare_ruddit_dataset():
    ruddit_train_df = pd.read_csv('/content/drive/MyDrive/kaggle-project/archive/Dataset/ruddit_with_text.csv')
    #ruddit_train_df = ruddit_train_df[0:200]
    #ruddit_train_df, ruddit_test_df = train_test_split(ruddit_df, test_size=0.2)
    
    ruddit_train = Dataset.from_pandas(ruddit_train_df)
    #ruddit_test = Dataset.from_pandas(ruddit_test_df)
    
    ruddit_train = ruddit_train.rename_column("offensiveness_score", "label")
    #ruddit_test = ruddit_test.rename_column("offensiveness_score", "label")

    # ruddit_train.set_format("torch")
    # ruddit_test.set_format("torch")
    
    ruddit_tokenized_train = ruddit_train.map(tokenize_function, batched=True)
    #ruddit_tokenized_test = ruddit_test.map(tokenize_function, batched=True)

    ruddit_tokenized_train.set_format("torch")
    #ruddit_tokenized_test.set_format("torch")
    
    return ruddit_tokenized_train

def prepare_validation_set():
    validation_data = pd.read_csv('/content/drive/MyDrive/kaggle-project/validation_data.csv')
    validation_data = validation_data[:7500]
    #validation_data = validation_data[:100]

    validation_comments = pd.DataFrame()

    list1 = list(map(lambda k: str(k) + '_1', validation_data.index.to_list()))
    list2 = list(map(lambda k: str(k) + '_2', validation_data.index.to_list()))

    scores = [0.0]*len(list1)
    scores.extend([1.0]*len(list1))

    list1.extend(list2)

    list1_comments = validation_data['less_toxic'].to_list()
    list2_comments = validation_data['more_toxic'].to_list()
    list1_comments.extend(list2_comments)

    validation_comments['id'] = list1
    validation_comments['txt'] = list1_comments
    validation_comments['label'] = scores

    validation_comments_ds = Dataset.from_pandas(validation_comments)
    validation_comments_ds = validation_comments_ds.map(tokenize_function, batched=True)

    validation_comments_ds.set_format("torch")

    return validation_comments_ds


In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report

def compute_metrics(pred):
    validation_data = pd.read_csv('/content/drive/MyDrive/kaggle-project/validation_data.csv')
    validation_data = validation_data[:7500]
    #validation_data = validation_data[:100]

    preds = pred.predictions

    test = preds[7500:15000].copy()
    #test = preds[30108:60216].copy()
    #test = preds[100:200].copy()

    validation_data['less_toxic_score'] = preds[0:7500]
    #validation_data['less_toxic_score'] = preds[0:100]
    #validation_data['less_toxic_score'] = preds[0:30108]
    validation_data['more_toxic_score'] = test

    validation_data['validation_score'] = validation_data['more_toxic_score'] > validation_data['less_toxic_score']
    
    #print(validation_data[validation_data['validation_score'] == True].shape[0])

    return {
        'accuracy': validation_data[validation_data['validation_score'] == True].shape[0] / validation_data.shape[0]
    }

In [ ]:
train_dataset = prepare_ruddit_dataset()

train_dataset.to_pandas()

  0%|          | 0/6 [00:00<?, ?ba/s]

,attention_mask,comment_id,input_ids,label,post_id,token_type_ids,txt,url
0,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",cza1q49,"[101, 1028, 1996, 4489, 1999, 2779, 16565, 209...",-0.083,42g75o,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",> The difference in average earnings between m...,https://www.reddit.com/r/changemyview/comments...
1,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",cza1wdh,"[101, 1996, 10661, 2003, 2008, 1996, 1000, 657...",-0.022,42g75o,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","The myth is that the ""gap"" is entirely based o...",https://www.reddit.com/r/changemyview/comments...
2,"[1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",cza23qx,"[101, 1031, 17159, 1033, 102, 0, 0, 0, 0, 0, 0...",0.167,42g75o,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[deleted],https://www.reddit.com/r/changemyview/comments...
3,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",cza2bw8,"[101, 1996, 23617, 2003, 2008, 2308, 2131, 382...",-0.146,42g75o,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",The assertion is that women get paid less for ...,https://www.reddit.com/r/changemyview/comments...
4,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",cza2iji,"[101, 2017, 2056, 1999, 1996, 6728, 2008, 1005...",-0.083,42g75o,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",You said in the OP that's not what they're mea...,https://www.reddit.com/r/changemyview/comments...
...,...,...,...,...,...,...,...,...
5833,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",f0i0mqp,"[101, 2027, 2323, 2069, 8292, 29577, 2477, 200...",0.064,cu67co,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",They should only censor things that talk badly...,https://i.redd.it/kfsmqzxae3i31.jpg/f0i0mqp/
5834,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",f80wlxq,"[101, 1028, 1998, 2028, 1997, 2068, 2003, 1037...",0.458,cganu1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",> and one of them is a woman. \n\nOH SHIT we b...,https://www.reddit.com/r/worldpolitics/comment...
5835,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, ...",f8uksbp,"[101, 2129, 2003, 2023, 14937, 2004, 2149, 433...",-0.292,cu67co,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",how is this flared as US politics,https://i.redd.it/kfsmqzxae3i31.jpg/f8uksbp/
5836,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",fa6nc1r,"[101, 2111, 1999, 4291, 4290, 2442, 5630, 2065...",0.333,cganu1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",People in Hong Kong must decide if they are go...,https://www.reddit.com/r/worldpolitics/comment...


In [ ]:
eval_dataset = prepare_validation_set()

eval_dataset.to_pandas()

  0%|          | 0/15 [00:00<?, ?ba/s]

,attention_mask,id,input_ids,label,token_type_ids,txt
0,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, ...",0_1,"[101, 2023, 3720, 19237, 15854, 15854, 15854, ...",0.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",This article sucks \n\nwoo woo wooooooo
1,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",1_1,"[101, 1000, 1998, 2748, 1010, 2111, 2323, 6807...",0.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","""And yes, people should recognize that but the..."
2,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",2_1,"[101, 2530, 2865, 1029, 9805, 2361, 1010, 2138...",0.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","Western Media?\n\nYup, because every crime in..."
3,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",3_1,"[101, 1998, 2017, 3718, 2009, 999, 2017, 15903...",0.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",And you removed it! You numbskull! I don't car...
4,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",4_1,"[101, 5437, 2100, 12436, 20876, 2630, 8180, 97...",0.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",smelly vagina \n\nBluerasberry why don't you ...
...,...,...,...,...,...,...
14995,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",7495_2,"[101, 1027, 5236, 23246, 2009, 1005, 1055, 605...",1.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",=Stupid Texans\n\nIt's funny how this is a par...
14996,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",7496_2,"[101, 1027, 5236, 23246, 2009, 1005, 1055, 605...",1.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",=Stupid Texans\n\nIt's funny how this is a par...
14997,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",7497_2,"[101, 1027, 5236, 23246, 2009, 1005, 1055, 605...",1.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",=Stupid Texans\n\nIt's funny how this is a par...
14998,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",7498_2,"[101, 2017, 4364, 2024, 8239, 2561, 22052, 201...",1.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",You guys are fucking total assholes... TOTAL. ...


In [ ]:
training_args = TrainingArguments(
    output_dir = './outputs',
    per_device_eval_batch_size = 128,
    seed = 42,
    num_train_epochs=5,
    logging_steps=500,
    eval_steps=500,
    evaluation_strategy='steps',
    load_best_model_at_end=True,
    metric_for_best_model='accuracy',
)

trainer = Trainer(
    model=model, args=training_args, train_dataset=train_dataset, eval_dataset=eval_dataset, compute_metrics=compute_metrics
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: comment_id, url, post_id, txt.
***** Running training *****
  Num examples = 5838
  Num Epochs = 5
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 3650


Step,Training Loss,Validation Loss,Accuracy
500,0.022700,0.324472,0.664533
1000,0.026500,0.340115,0.679067
1500,0.018400,0.321311,0.671867
2000,0.012100,0.313569,0.683333
2500,0.009700,0.298422,0.679600
3000,0.006600,0.311532,0.678267
3500,0.005800,0.305862,0.675467


The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: txt, id.
***** Running Evaluation *****
  Num examples = 15000
  Batch size = 128
Saving model checkpoint to ./outputs/checkpoint-500
Configuration saved in ./outputs/checkpoint-500/config.json
Model weights saved in ./outputs/checkpoint-500/pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: txt, id.
***** Running Evaluation *****
  Num examples = 15000
  Batch size = 128
Saving model checkpoint to ./outputs/checkpoint-1000
Configuration saved in ./outputs/checkpoint-1000/config.json
Model weights saved in ./outputs/checkpoint-1000/pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: txt, id.
***** Running Evaluation **

TrainOutput(global_step=3650, training_loss=0.014167767338556786, metrics={'train_runtime': 3639.6672, 'train_samples_per_second': 8.02, 'train_steps_per_second': 1.003, 'total_flos': 7680142748436480.0, 'train_loss': 0.014167767338556786, 'epoch': 5.0})

In [ ]:
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: txt, id.
***** Running Evaluation *****
  Num examples = 15000
  Batch size = 128


{'epoch': 5.0,
 'eval_accuracy': 0.6833333333333333,
 'eval_loss': 0.31356939673423767,
 'eval_runtime': 268.0504,
 'eval_samples_per_second': 55.96,
 'eval_steps_per_second': 0.44}

In [ ]:
trainer.save_model('/content/drive/MyDrive/kaggle-project/68-percent-model/')


Saving model checkpoint to /content/drive/MyDrive/kaggle-project/68-percent-model/
Configuration saved in /content/drive/MyDrive/kaggle-project/68-percent-model/config.json
Model weights saved in /content/drive/MyDrive/kaggle-project/68-percent-model/pytorch_model.bin
